In [ ]:
%cd ..

In [ ]:
from apperception.database import database
from apperception.world import empty_world
from apperception.utils import F
from apperception.predicate import camera, objects

In [ ]:
from optimized_ingestion.pipeline import Pipeline
import optimized_ingestion

In [ ]:
o1 = objects[0]
o2 = objects[1]
c = camera

predicate = (
    (o1.id != o2.id) &
    (F.like(o1.type, 'car') | F.like(o1.type, 'truck')) &
    (F.like(o2.type, 'car') | F.like(o2.type, 'truck')) &
    F.angle_between(F.facing_relative(c.ego, F.road_direction(c.ego)), -15, 15) &
    (F.distance(c.ego, o1.trans@c.time) < 50) &
    (F.view_angle(o1.trans@c.time, c.ego) < 70 / 2.0) &
    (F.distance(c.ego, o2.trans@c.time) < 50) &
    (F.view_angle(o2.trans@c.time, c.ego) < 70 / 2.0) &
    F.contains_all('intersection', [o1.trans, o2.trans]@c.time) &
    F.angle_between(F.facing_relative(o1.trans@c.time, c.ego), 40, 135) &
    F.angle_between(F.facing_relative(o2.trans@c.time, c.ego), -135, -50) &
    (F.min_distance(c.ego, 'intersection') < 10) &
    F.angle_between(F.facing_relative(o1.trans@c.time, o2.trans@c.time), 100, -100)
)

In [ ]:
from optimized_ingestion.stages.in_view import InView
from optimized_ingestion.stages.decode_frame.decode_frame import DecodeFrame
from optimized_ingestion.stages.detection_2d.yolo_detection import YoloDetection
from optimized_ingestion.stages.detection_2d.object_type_filter import ObjectTypeFilter
from optimized_ingestion.stages.detection_3d.from_2d_and_road import From2DAndRoad
from optimized_ingestion.stages.detection_3d.from_2d_and_depth import From2DAndDepth
from optimized_ingestion.stages.depth_estimation import DepthEstimation
from optimized_ingestion.stages.detection_estimation import DetectionEstimation
from optimized_ingestion.stages.tracking_2d.strongsort import StrongSORT


def create_pipeline(
    predicate,
    in_view=True,
    object_filter=True,
    groundtruth_detection=False,
    geo_depth=True,
    detection_estimation=True,
    strongsort=False,
):
    pipeline = Pipeline()

    # In-View Filter
    if in_view:
        # TODO: view angle and road type should depends on the predicate
        pipeline.add_filter(InView(50, 'intersection'))

    # Decode
    pipeline.add_filter(DecodeFrame())

    # 2D Detection
    if groundtruth_detection:
        pass
    else:
        pipeline.add_filter(YoloDetection())

    # Object Filter
    if object_filter:
        # TODO: filter objects based on predicate
        pipeline.add_filter(ObjectTypeFilter([]))

    # 3D Detection
    if geo_depth:
        pipeline.add_filter(From2DAndRoad())
    else:
        pipeline.add_filter(DepthEstimation())
        pipeline.add_filter(From2DAndDepth())

    # Detection Estimation
    if detection_estimation:
        pipeline.add_filter(DetectionEstimation())

    # Tracking
    pipeline.add_filter(StrongSORT(cache=strongsort))


In [ ]:
p_inview = create_pipeline(
    predicate,
    in_view=True,
    object_filter=False,
    geo_depth=False,
    detection_estimation=False
)

p_objfilter = create_pipeline(
    predicate,
    in_view=False,
    object_filter=True,
    geo_depth=False,
    detection_estimation=False
)

p_geo = create_pipeline(
    predicate,
    in_view=False,
    object_filter=False,
    geo_depth=True,
    detection_estimation=False
)

p_opt = create_pipeline(
    predicate,
    in_view=True,
    object_filter=True,
    geo_depth=True,
    detection_estimation=False
)

p_opt_de = create_pipeline(
    predicate,
    in_view=True,
    object_filter=True,
    geo_depth=True,
    detection_estimation=True
)

pipelines = [p_inview, p_objfilter, p_geo, p_opt, p_opt_de]